In [1]:
import numpy as np
import pandas as pd
import random
import itertools
import time
import operator
from operator import itemgetter
import math
import scipy.stats as ss
import pprint as pp

In [2]:
file_path = r"C:\Users\91861\Downloads\CAB_TR_RGP_Datasets 2023(1).xlsx"

#Load demand data
CAB10_flow_df= pd.read_excel(file_path, skiprows=2, nrows=10, usecols="B:K", header=None)
CAB10_flow_df.index += 1
CAB10_flow_df.columns += 1

#Load unit cost data
CAB10_cost_df = pd.read_excel(file_path, skiprows = 18, nrows=10, usecols="B:K", header=None)
CAB10_cost_df.index += 1
CAB10_cost_df.columns += 1


CAB25_flow_df= pd.read_excel(file_path, skiprows=34, nrows=25, usecols="B:Z", header=None)
CAB25_flow_df.index += 1
CAB25_flow_df.columns += 1

CAB25_cost_df = pd.read_excel(file_path, skiprows = 62, nrows=25, usecols="B:Z", header=None)
CAB25_cost_df.index += 1
CAB25_cost_df.columns += 1

TR55_flow_df= pd.read_excel(file_path, sheet_name=1, skiprows=5, nrows=55, usecols="D:BF", header=None)
TR55_flow_df.index += 1
TR55_flow_df.columns += 1

TR55_cost_df = pd.read_excel(file_path,sheet_name=1, skiprows = 63, nrows=55, usecols="D:BF", header=None)
TR55_cost_df.index += 1
TR55_cost_df.columns += 1

TR81_flow_df= pd.read_excel(file_path, sheet_name=2, skiprows=2, nrows=81, usecols="D:CF", header=None)
TR81_flow_df.index += 1
TR81_flow_df.columns += 1

TR81_cost_df = pd.read_excel(file_path,sheet_name=2, skiprows = 86, nrows=81, usecols="D:CF", header=None)
TR81_cost_df.index += 1
TR81_cost_df.columns += 1

RGP100_flow_df= pd.read_excel(file_path, sheet_name=3, skiprows=3, nrows=100, usecols="B:CW", header=None)
RGP100_flow_df.index += 1
RGP100_flow_df.columns += 1

RGP100_cost_df = pd.read_excel(file_path,sheet_name=3, skiprows =3, nrows=100, usecols="B:CW", header=None)
RGP100_cost_df.index += 1
RGP100_cost_df.columns += 1

In [3]:
#Transform csv dataframe to list
CAB10_flow = CAB10_flow_df.to_numpy()  
CAB10_cost = CAB10_cost_df.to_numpy()

CAB25_flow = CAB25_flow_df.to_numpy()  
CAB25_cost = CAB25_cost_df.to_numpy()

TR55_flow = TR55_flow_df.to_numpy()  
TR55_cost = TR55_cost_df.to_numpy()

TR81_flow = TR81_flow_df.to_numpy()  
TR81_cost = TR81_cost_df.to_numpy()

RGP100_flow = RGP100_flow_df.to_numpy()  
RGP100_cost = RGP100_cost_df.to_numpy()

In [4]:
CAB10_flow

array([[    0,  6469,  7629, 20036,  4690,  6194, 11688,  2243,  8857,
         7248],
       [ 6469,     0, 12999, 13692,  3322,  5576,  3878,  3202,  6699,
         4198],
       [ 7629, 12999,     0, 35135,  5956, 14121,  5951,  5768, 16578,
         4242],
       [20036, 13692, 35135,     0, 19094, 35119, 21423, 27342, 51341,
        15826],
       [ 4690,  3322,  5956, 19094,     0,  7284,  3102,  1562,  7180,
         1917],
       [ 6194,  5576, 14121, 35119,  7284,     0,  5023,  3512, 10419,
         3543],
       [11688,  3878,  5951, 21423,  3102,  5023,     0, 11557,  6479,
        34261],
       [ 2243,  3202,  5768, 27342,  1562,  3512, 11557,     0,  5615,
         7095],
       [ 8857,  6699, 16578, 51341,  7180, 10419,  6479,  5615,     0,
         4448],
       [ 7248,  4198,  4242, 15826,  1917,  3543, 34261,  7095,  4448,
            0]], dtype=int64)

In [5]:
case1_opt_sol = 740999439
case2_opt_sol = 8374475034

In [6]:
# Create a function to get the first row particle
def get_first_row_particle(n_node, x_min, x_max):
    return [round(x_min + (x_max - x_min) * random.uniform(0, 1), 2) for _ in range(n_node)]

In [7]:
# Create a function to sort a particle by its dimension's value (Ascending)
def get_dimension_rank(particle):
    return [int(dimension) for dimension in ss.rankdata(particle)]

In [8]:
# Select the first p particles as the hub
def get_hub_allocation(dimension_rank, n_hub):
    return dict(zip(range(1, n_hub+1), dimension_rank[:n_hub]))

In [9]:
# Create a function to get the second row particle
def get_second_row_particle(n_node, n_hub, epsilon):
    return [round(random.uniform(1, n_hub+1-epsilon), 2) for _ in range(1, n_node+1)]

In [10]:
# Create a function to reallocate spoke to its cheapest hub
def dimension_reallocation(particle, n_node, cost_matrix, unique_hub):
    spoke_index_list = list(range(n_node))
    
    # Ensure the allocation is still within the number of nodes
    particle = [dimension if dimension <= n_node else random.randint(0, n_node-1) for dimension in particle]
    
    # Ensure we obtain feasible solutions only
    for hub in unique_hub:
        if particle[hub-1] != hub:
            particle[hub-1] = hub
    
    spoke_hub_cost = {}
    for index, spoke in enumerate(spoke_index_list):
        spoke_hub_cost[spoke+1] = [(cost_matrix[index][hub-1], hub) for hub in unique_hub]

    spoke_hub_mapping_with_lowest_cost = {}
    for spoke, hub_cost in spoke_hub_cost.items():
        min_cost = min([cost for cost, _ in hub_cost])
        hub = [h for c, h in hub_cost if c == min_cost][0]
        spoke_hub_mapping_with_lowest_cost[spoke] = hub 

    spoke_hub_index_mapping_with_lowest_cost = [(index, hub-1) for index, spoke in enumerate(particle) if spoke not in unique_hub
                                                for hub, cost in spoke_hub_cost[spoke].items() if cost == min([c for c, _ in spoke_hub_cost[spoke]])]

    solution = particle.copy()
    for index, hub_index in spoke_hub_index_mapping_with_lowest_cost:
        solution[index] = hub_index+1

    return solution

In [11]:
# Create a function to repair a particle in case the total hub generated != the given number of hub
import random
import itertools
from typing import List

def repair_particle(particle: List[int], n_node: int, n_hub: int, cost_matrix: List[List[float]]) -> List[int]:

    hubs_particle = []

    for i, dim in enumerate(particle):
        if dim > n_node:
            particle[i] = random.choice(range(n_node))
    
    unique_hub = [hub for hub in list(set(particle)) if hub != 0]
    particle_dict = dict(zip(range(n_node), particle))

    for hub in unique_hub:
        if particle[hub-1] != hub:
            particle[hub-1] = hub

    if len(unique_hub) == n_hub:
        for i, dim in enumerate(particle):
            if dim == 0:
                particle[i] = random.choice(unique_hub)
        particle_repaired = dimension_reallocation(particle, n_node, cost_matrix, unique_hub)
            
    elif len(unique_hub) > n_hub:
        repaired_unique_hub = random.choice(list(itertools.combinations(unique_hub, n_hub)))
        for i, dim in enumerate(particle):
            if dim == 0 or particle_dict[i] not in repaired_unique_hub:
                particle[i] = random.choice(repaired_unique_hub)
        particle_repaired = dimension_reallocation(particle, n_node, cost_matrix, repaired_unique_hub)
        
    elif len(unique_hub) < n_hub:
        new_hub_required = n_hub - len(unique_hub)
        possible_new_hub = set(range(1, n_node+1)) - set(unique_hub)
        random_new_hub = random.choices(list(possible_new_hub), k=new_hub_required)
        repaired_unique_hub = unique_hub + random_new_hub
        for i, dim in enumerate(particle):
            if dim == 0 or particle_dict[i] not in repaired_unique_hub:
                particle[i] = random.choice(repaired_unique_hub)
        particle_repaired = dimension_reallocation(particle, n_node, cost_matrix, repaired_unique_hub)
        
    return particle_repaired


In [12]:
# Create a function 
def get_initial_row_particle_velocity(particle, theta, n_node, n_hub):
    v_min, v_max = 0, n_node * n_hub * theta
    return [round(v_min + (v_max - v_min) * random.uniform(0, 1), 2) for _ in range(len(particle))]


In [13]:
def get_first_row_particle_updated_velocity(n_node, current_position, current_velocity, particle_best, global_best, inertia_factor, C):
    r1r2 = [random.uniform(0, 1) for _ in range(n_node)]
    new_velocity = [
        (inertia_factor * current_velocity[i]) + C * r1r2[i] * (particle_best[i] - current_position[i]) + C * r1r2[i] * (global_best[i] - current_position[i])
        for i in range(n_node)
    ]
    return [round(velocity, 2) for velocity in new_velocity]


In [14]:
def get_second_row_particle_updated_velocity(n_node, current_position, current_velocity, particle_best, global_best, inertia_factor, C):
    r1r2 = [random.uniform(0, 1) for _ in range(n_node)]
    new_velocity = [
        (inertia_factor * current_velocity[i]) + C * r1r2[i] * (particle_best[i] - current_position[i]) + C * r1r2[i] * (global_best[i] - current_position[i])
        for i in range(n_node)
    ]
    return [round(velocity, 2) for velocity in new_velocity]


In [15]:
# Create a function to compute the total distance in the network
def total_hub_flow(solution, node, flow_matrix):

    n_hub = max(solution)
    hub_spoke = {hub: [i for i, val in enumerate(solution) if val == hub and i != hub-1] for hub in range(1, n_hub+1)}
    
    hub_flows = {hub: [flow_matrix[hub-1]] + [flow_matrix[spoke_index] for spoke_index in hub_spoke[hub]] for hub in range(1, n_hub+1)}
    
    total_hub_flow = {hub: np.sum(hub_flows[hub]) for hub in range(1, n_hub+1)}
    
    return total_hub_flow

In [16]:
def alpha(flow):
    unit_flow_cost = 0
    
    if flow < 50000:
        unit_flow_cost = 0 + 1*flow
    elif flow < 100000:
        unit_flow_cost = 10000 + 0.8*flow
    elif flow < 200000:
        unit_flow_cost = 30000 + 0.6*flow
    else:
        unit_flow_cost = 70000 + 0.4*flow
    return unit_flow_cost

In [17]:
def total_network_cost(solution, node, flow_matrix, cost_matrix):
    total_cost = 0   
    #Identify the hubs
    hubs = set(solution)   
   #Calculate collection and distribution cost
    for i in range(0, len(solution)):
        for j in range(0, len(solution)):
            total_cost += flow_matrix[i][j]*(cost_matrix[i][solution[i]-1] + cost_matrix[solution[j]-1][j])   
   #Calculate inter-hub cost
    for k in hubs:
        for m in hubs:
            if k != m:
                interhub_flow = 0
                for i in range(0, len(solution)):
                    for j in range(0, len(solution)):
                        if solution[i] == k and solution[j] == m:
                            interhub_flow += flow_matrix[i][j]                      #Calculate inter-hub flow
                
                total_cost +=  alpha(interhub_flow)*cost_matrix[k-1][m-1]       #Calculate inter-hub cost
    
    return total_cost

In [18]:
a=[1,2,3,1,2,3,1,2,3,1]

In [19]:
total_network_cost(a,10,CAB10_flow,CAB10_cost)

1449966595.4143999

In [20]:
# Create a function to compute the total network cost
def compute_particle_cost(particle, node, flow_matrix, cost_matrix, alpha):
    hub_flows = total_hub_flow(particle, node, flow_matrix)
    total_flow_cost = total_network_cost(particle, node, flow_matrix, cost_matrix)
    return round(total_flow_cost, 2)

In [21]:
compute_particle_cost(a,10,CAB10_flow,CAB10_cost, alpha)

1449966595.41

In [22]:
def pso(n_node, n_hub, swarm_size, flow_matrix, cost_matrix, epsilon, theta, inertia_factor, C):
    iteration_count = 0
    x_min = 0
    x_max = n_node*n_hub
    while iteration_count < swarm_size: # Stopping criteria: The number of iteration = Swarm size
        
        #======================================= FIRST ITERATION =======================================#
        if iteration_count == 0:
            print(f'Initial iteration is starting...')
            # Declare inputs
            initial_row1_particles = []
            initial_row2_particles = []
            row1_particles_rank = []
            selected_hubs_dict = [] 
            initial_swarm_discrete = []
            initial_swarm_repaired = []
            initial_swarm_cost = []

            row2_updated_velocity = []
            first_row_new_position = []
            second_row_new_position = []

            # Step 1: Populate first row and second row particles
            counter = 0
            while counter < swarm_size:
                particle1 = get_first_row_particle(n_node, x_min, x_max)
                initial_row1_particles.append(particle1)
                particle2 = get_second_row_particle(n_node, n_hub, epsilon)
                initial_row2_particles.append(particle2)
                counter += 1

            row1_current_position = initial_row1_particles
            row2_current_position = initial_row2_particles

            for particle_index in list(range(0, swarm_size)):
                # Step 2: Rank first row particle
                row1_rank = get_dimension_rank(row1_current_position[particle_index])
                row1_particles_rank.append(row1_rank)

                # Step 3: Get the selected hubs from the first row particle     
                selected_hubs = get_hub_allocation(row1_particles_rank[particle_index], n_hub)
                selected_hubs_dict.append(selected_hubs)

                # Step 4: Allocate the hub to the second row particle to generate initial swarm result in discrete values
                particle = []
                for dimension_index in list(range(0, n_node)):
                    row2_dim_int = math.floor(row2_current_position[particle_index][dimension_index])
                    allocated_hub = selected_hubs_dict[particle_index][row2_dim_int]
                    particle.append(allocated_hub)
                initial_swarm_discrete.append(particle)

                # Step 5: Repair particles in the swarm
                particle_repaired = repair_particle(initial_swarm_discrete[particle_index], n_node, n_hub, cost_matrix)
                initial_swarm_repaired.append(particle_repaired)

                # Step 6: Compute fitness
                cost = compute_particle_cost(initial_swarm_repaired[particle_index], n_node, flow_matrix, cost_matrix, alpha)
                initial_swarm_cost.append(cost)

            # Step 7: Declare particle best and global best
                row1_particle_best = list(zip(initial_swarm_cost, row1_current_position))
                row1_global_best = min(row1_particle_best, key=itemgetter(0))
                row2_particle_best = list(zip(initial_swarm_cost, row2_current_position))
                row2_global_best = min(row2_particle_best, key=itemgetter(0))
                
            global_best_discrete_position = []
            for index in list(range(0, swarm_size)):
                if row1_particle_best[index][0] == row1_global_best[0]:
                    global_best_discrete_position = initial_swarm_repaired[index]
                else:
                    pass
                
            # Step 8: Compute initial velocity
            row1_initial_velocity = []
            row2_initial_velocity = []    
            for particle_index in list(range(0, swarm_size)):                
                row1_initial_velocity_temp = get_initial_row_particle_velocity(row1_current_position[particle_index], n_node, n_hub, theta)
                row1_initial_velocity.append(row1_initial_velocity_temp)

                row2_initial_velocity_temp = get_initial_row_particle_velocity(row2_current_position[particle_index], n_node, n_hub, theta)
                row2_initial_velocity.append(row2_initial_velocity_temp)
                
            row1_current_velocity = row1_initial_velocity
            row2_current_velocity = row2_initial_velocity
            
            # Step 9: Compute updated velocity
            row1_updated_velocity = []
            row1_global_best_position = row1_global_best[1]
            
            row2_updated_velocity = []
            row2_global_best_position = row2_global_best[1]
            
            for particle_index in list(range(0, swarm_size)):
                row1_particle_best_position = row1_particle_best[particle_index][1]
                row1_velocity_temp = get_first_row_particle_updated_velocity(n_node, row1_current_position[particle_index], row1_current_velocity[particle_index], row1_particle_best_position, row1_global_best_position, inertia_factor, C)
                row1_updated_velocity.append(row1_velocity_temp)

                row2_particle_best_position = row2_particle_best[particle_index][1]
                row2_velocity_temp = get_second_row_particle_updated_velocity(n_node, row2_current_position[particle_index], row2_current_velocity[particle_index], row2_particle_best_position, row2_global_best_position, inertia_factor, C)
                row2_updated_velocity.append(row2_velocity_temp)
                
            # Step 10: Update particle position
            row1_new_swarm_position_list = []
            for particle_index in list(range(0, swarm_size)):
                row1_new_particle_position = []
                for dim_index in list(range(0, n_node)):
                    row1_new_dimension_temp = row1_current_position[particle_index][dim_index] + row1_updated_velocity[particle_index][dim_index]
                    row1_new_dimension_temp = round(row1_new_dimension_temp, 2)
                    row1_new_particle_position.append(row1_new_dimension_temp)
                row1_new_swarm_position_list.append(row1_new_particle_position)

            row2_new_swarm_position_list = []
            for particle_index in list(range(0, swarm_size)):
                row2_new_particle_position = []
                for dim_index in list(range(0, n_node)):
                    row2_new_dimension_temp = row2_current_position[particle_index][dim_index] + row2_updated_velocity[particle_index][dim_index]
                    row2_new_dimension_temp = round(row2_new_dimension_temp, 2)
                    row2_new_particle_position.append(row2_new_dimension_temp)
                row2_new_swarm_position_list.append(row2_new_particle_position)  
                
            # Step 11: Re-declare the particle best and global best for the next iteration
            old_row1_particle_best = row1_particle_best
            old_row2_particle_best = row2_particle_best
            old_global_best = row1_global_best[0]
            
         
        #======================================= SUBSEQUENT ITERATION =======================================#
        elif iteration_count > 0:
            # Step 1: Populate second row particles
            new_row2_particles = []
            counter = 0
            while counter < swarm_size:
                particle2 = get_second_row_particle(n_node, n_hub, epsilon)
                new_row2_particles.append(particle2)
                counter += 1
            
            # Declare current positions
            row1_current_position = row1_new_swarm_position_list
            row2_current_position = new_row2_particles
            row1_particles_rank = []
            selected_hubs_dict = []
            new_swarm_discrete = []
            new_swarm_repaired = []
            new_swarm_cost = []
            row1_particle_best = []
            
            for particle_index in list(range(0, swarm_size)):
                # Step 2: Rank first row particle
                row1_rank = get_dimension_rank(row1_current_position[particle_index])
                row1_particles_rank.append(row1_rank)

                # Step 3: Get the selected hubs from the first row particle     
                selected_hubs = get_hub_allocation(row1_particles_rank[particle_index], n_hub)
                selected_hubs_dict.append(selected_hubs)

                # Step 4: Allocate the hub to the second row particle to generate new swarm in discrete values
                particle = []
                for dimension_index in list(range(0, n_node)):
                    row2_dim_int = math.floor(row2_current_position[particle_index][dimension_index])
                    allocated_hub = selected_hubs_dict[particle_index][row2_dim_int]
                    particle.append(allocated_hub)
                new_swarm_discrete.append(particle)

                # Step 5: Repair particles in the swarm
                particle_repaired = repair_particle(new_swarm_discrete[particle_index], n_node, n_hub, cost_matrix)
                new_swarm_repaired.append(particle_repaired)

                # Step 6: Compute fitness
                cost = compute_particle_cost(new_swarm_repaired[particle_index], n_node, flow_matrix, cost_matrix, alpha)
                new_swarm_cost.append(cost)

            # Step 7: Declare particle best and global best  
                if new_swarm_cost[particle_index] < old_row1_particle_best[particle_index][0]:
                    row1_particle_best.append((new_swarm_cost[particle_index], row1_current_position[particle_index]))
                else:
                    row1_particle_best.append(old_row1_particle_best[particle_index])
                    
                if new_swarm_cost[particle_index] < old_row2_particle_best[particle_index][0]:
                    row2_particle_best.append((new_swarm_cost[particle_index], row2_current_position[particle_index]))
                else:
                    row2_particle_best.append(old_row2_particle_best[particle_index])    
            
            row1_global_best = min(row1_particle_best, key=itemgetter(0))
            row2_global_best = min(row1_particle_best, key=itemgetter(0))
            
            global_best_discrete_position = []
            for index in list(range(0, swarm_size)):
                if row1_particle_best[index][0] == row1_global_best[0]:
                    global_best_discrete_position = new_swarm_repaired[index]
                else:
                    pass
                
            # Step 8 is skipped because we no longer have to compute the initial velocity
            
            # Step 9: Compute updated velocity
            row1_updated_velocity = []
            row1_global_best_position = row1_global_best[1]
            
            row2_updated_velocity = []
            row2_global_best_position = row2_global_best[1]
            
            for particle_index in list(range(0, swarm_size)):
                row1_particle_best_position = row1_particle_best[particle_index][1]
                row1_velocity_temp = get_first_row_particle_updated_velocity(n_node, row1_current_position[particle_index], row1_current_velocity[particle_index], row1_particle_best_position, row1_global_best_position, inertia_factor, C)
                row1_updated_velocity.append(row1_velocity_temp)

                row2_particle_best_position = row2_particle_best[particle_index][1]
                row2_velocity_temp = get_second_row_particle_updated_velocity(n_node, row2_current_position[particle_index], row2_current_velocity[particle_index], row2_particle_best_position, row2_global_best_position, inertia_factor, C)
                row2_updated_velocity.append(row2_velocity_temp) 
                
            # Step 10: Update particle position
            row1_new_swarm_position_list = []
            for particle_index in list(range(0, swarm_size)):
                row1_new_particle_position = []
                for dim_index in list(range(0, n_node)):
                    row1_new_dimension_temp = row1_current_position[particle_index][dim_index] + row1_updated_velocity[particle_index][dim_index]
                    row1_new_dimension_temp = round(row1_new_dimension_temp, 2)
                    row1_new_particle_position.append(row1_new_dimension_temp)
                row1_new_swarm_position_list.append(row1_new_particle_position)

            row2_new_swarm_position_list = []
            for particle_index in list(range(0, swarm_size)):
                row2_new_particle_position = []
                for dim_index in list(range(0, n_node)):
                    row2_new_dimension_temp = row2_current_position[particle_index][dim_index] + row2_updated_velocity[particle_index][dim_index]
                    row2_new_dimension_temp = round(row2_new_dimension_temp, 2)
                    row2_new_particle_position.append(row2_new_dimension_temp)
                row2_new_swarm_position_list.append(row2_new_particle_position)     
            
            # Step 11: Re-declare the particle best and global best for the next iteration
            old_row1_particle_best = row1_particle_best
            old_row2_particle_best = row2_particle_best
            old_global_best = row1_global_best[0]

        iteration_count += 1
    
    # If all iterations are completed, compute the global best (total network cost), retrieve best configuration of the hub-and-spoke network, and the retrieve selected hub
    global_best = row1_global_best[0]
    best_config = global_best_discrete_position
    selected_hubs = list(set(best_config))
    global_best_solution = [global_best, best_config, selected_hubs]
    
    print(f'Global best value: {global_best}')
    print(f'Global best config: {best_config}')
    return global_best_solution

# 1) CAB  n=10, p=3 

In [23]:
%%time

start_time = time.time()

n_node=10
n_hub=3
swarm_size=100
epsilon=0.01
x_min=0
x_max = n_node*n_hub
flow_matrix = CAB10_flow
cost_matrix = CAB10_cost
theta = 0.1
inertia_factor = 1
C = 2

global_minimum_solution_list = []
replication = 0
n_replication = 10

while replication < n_replication:
    PSO_result = pso(n_node, n_hub, swarm_size, flow_matrix, cost_matrix, epsilon, theta, inertia_factor, C)
    global_minimum_solution_list.append(PSO_result)
    print(f'====================== REPLICATION {replication} IS FINISHED ======================\n')
    print('\n')
    replication += 1
    
case1_df = pd.DataFrame(global_minimum_solution_list, columns=['cost','node','unique_hub'])
avg_tnc = round(case1_df["cost"].mean(),2)
best_tnc = round(case1_df["cost"].min(),2)
gap = round((best_tnc - case1_opt_sol)/best_tnc*100,2)
avg_cpu_time = round((time.time() - start_time)/n_replication,2)

print(f'AVG CPU time: {avg_cpu_time} seconds')
print(f'Optimal solution answer: {case1_opt_sol}')
print(f'My optimal solution: {best_tnc}')
print(f'AVG optimal solution: {avg_tnc}')
print(f'Gap (%): {gap}%')

Initial iteration is starting...
Global best value: 740999438.94
Global best config: [5, 9, 7, 5, 5, 9, 7, 9, 9, 5]
====================== REPLICATION 0 IS FINISHED ======================



Initial iteration is starting...
Global best value: 750795242.36
Global best config: [7, 6, 5, 6, 5, 6, 7, 5, 6, 5]
====================== REPLICATION 1 IS FINISHED ======================



Initial iteration is starting...
Global best value: 750122927.98
Global best config: [1, 2, 7, 2, 1, 7, 7, 7, 1, 7]
====================== REPLICATION 2 IS FINISHED ======================



Initial iteration is starting...
Global best value: 747010706.97
Global best config: [2, 2, 2, 5, 5, 2, 7, 5, 7, 2]
====================== REPLICATION 3 IS FINISHED ======================



Initial iteration is starting...
Global best value: 745421728.63
Global best config: [5, 5, 4, 4, 5, 5, 7, 7, 7, 5]
====================== REPLICATION 4 IS FINISHED ======================



Initial iteration is starting...
Global best 

# 2) CAB n=25, p=3

In [24]:
%%time

start_time = time.time()

n_node=25
n_hub=3
swarm_size= 50
epsilon=0.01
x_min=0
x_max = n_node*n_hub
flow_matrix = CAB25_flow
cost_matrix = CAB25_cost
theta = 0.1
inertia_factor = 1
C = 2

global_minimum_solution_list = []
replication = 0
n_replication = 10

while replication < n_replication:
    PSO_result = pso(n_node, n_hub, swarm_size, flow_matrix, cost_matrix, epsilon, theta, inertia_factor, C)
    global_minimum_solution_list.append(PSO_result)
    print(f'====================== REPLICATION {replication} IS FINISHED ======================\n')
    print('\n')
    replication += 1
    
case2_df = pd.DataFrame(global_minimum_solution_list, columns=['cost','node','unique_hub'])
avg_tnc = round(case2_df["cost"].mean(),2)
best_tnc = round(case2_df["cost"].min(),2)
gap = round((best_tnc - case2_opt_sol)/best_tnc*100,2)
avg_cpu_time = round((time.time() - start_time)/n_replication,2)

print(f'AVG CPU time: {avg_cpu_time} seconds')
print(f'Optimal solution answer: {case2_opt_sol}')
print(f'My optimal solution: {best_tnc}')
print(f'AVG optimal solution: {avg_tnc}')
print(f'Gap (%): {gap}%')

Initial iteration is starting...
Global best value: 11250046557.18
Global best config: [22, 8, 8, 22, 22, 22, 22, 8, 8, 22, 14, 8, 14, 14, 14, 14, 14, 14, 14, 8, 8, 22, 22, 14, 22]
====================== REPLICATION 0 IS FINISHED ======================



Initial iteration is starting...
Global best value: 11559413859.72
Global best config: [4, 22, 25, 4, 22, 4, 25, 4, 22, 22, 4, 4, 25, 22, 4, 4, 22, 25, 22, 25, 25, 22, 22, 4, 25]
====================== REPLICATION 1 IS FINISHED ======================



Initial iteration is starting...
Global best value: 11461768039.4
Global best config: [25, 25, 25, 13, 19, 13, 19, 13, 13, 25, 25, 25, 13, 13, 13, 13, 13, 19, 19, 19, 25, 19, 19, 19, 25]
====================== REPLICATION 2 IS FINISHED ======================



Initial iteration is starting...
Global best value: 11661520256.82
Global best config: [11, 25, 11, 25, 25, 6, 6, 6, 25, 25, 11, 6, 6, 11, 6, 25, 6, 6, 11, 11, 25, 25, 6, 25, 25]
====================== REPLICATION 3 IS FINISHED 

# 3) CAB n=25, p=5

In [25]:
%%time

start_time = time.time()

n_node=25
n_hub=5
swarm_size= 50
epsilon=0.01
x_min=0
x_max = n_node*n_hub
flow_matrix = CAB25_flow
cost_matrix = CAB25_cost
theta = 0.1
inertia_factor = 1
C = 2

global_minimum_solution_list = []
replication = 0
n_replication = 10

while replication < n_replication:
    PSO_result = pso(n_node, n_hub, swarm_size, flow_matrix, cost_matrix, epsilon, theta, inertia_factor, C)
    global_minimum_solution_list.append(PSO_result)
    print(f'====================== REPLICATION {replication} IS FINISHED ======================\n')
    print('\n')
    replication += 1
    
case2_df = pd.DataFrame(global_minimum_solution_list, columns=['cost','node','unique_hub'])
avg_tnc = round(case2_df["cost"].mean(),2)
best_tnc = round(case2_df["cost"].min(),2)
gap = round((best_tnc - case2_opt_sol)/best_tnc*100,2)
avg_cpu_time = round((time.time() - start_time)/n_replication,2)

print(f'AVG CPU time: {avg_cpu_time} seconds')
print(f'Optimal solution answer: {case2_opt_sol}')
print(f'My optimal solution: {best_tnc}')
print(f'AVG optimal solution: {avg_tnc}')
print(f'Gap (%): {gap}%')

Initial iteration is starting...
Global best value: 11662985180.58
Global best config: [19, 19, 24, 15, 15, 19, 19, 24, 12, 19, 18, 12, 18, 24, 15, 18, 15, 18, 19, 12, 12, 15, 24, 24, 19]
====================== REPLICATION 0 IS FINISHED ======================



Initial iteration is starting...
Global best value: 11683024730.53
Global best config: [15, 15, 4, 4, 5, 4, 7, 15, 7, 15, 4, 7, 7, 4, 15, 16, 4, 7, 16, 16, 7, 5, 4, 15, 5]
====================== REPLICATION 1 IS FINISHED ======================



Initial iteration is starting...
Global best value: 11701679951.19
Global best config: [7, 11, 14, 19, 19, 14, 7, 11, 14, 11, 11, 19, 14, 14, 21, 7, 14, 19, 19, 21, 21, 7, 14, 7, 21]
====================== REPLICATION 2 IS FINISHED ======================



Initial iteration is starting...
Global best value: 10987674898.57
Global best config: [22, 22, 3, 22, 5, 23, 22, 22, 22, 22, 3, 22, 17, 22, 23, 5, 17, 22, 23, 17, 5, 22, 23, 22, 5]
====================== REPLICATION 3 IS FINISHED =

# 4) TR n=55, p=3

In [26]:
%%time

start_time = time.time()

n_node=55
n_hub=5
swarm_size=100
epsilon=0.01
x_min=0
x_max = n_node*n_hub
flow_matrix = TR55_flow
cost_matrix = TR55_cost
theta = 0.1
inertia_factor = 1
C = 2

global_minimum_solution_list = []
replication = 0
n_replication = 10

while replication < n_replication:
    PSO_result = pso(n_node, n_hub, swarm_size, flow_matrix, cost_matrix, epsilon, theta, inertia_factor, C)
    global_minimum_solution_list.append(PSO_result)
    print(f'====================== REPLICATION {replication} IS FINISHED ======================\n')
    print('\n')
    replication += 1
    
case3_df = pd.DataFrame(global_minimum_solution_list, columns=['cost','node','unique_hub'])
avg_tnc = round(case3_df["cost"].mean(),2)
best_tnc = round(case3_df["cost"].min(),2)
#gap = round((best_tnc - case3_opt_sol)/best_tnc*100,2)
avg_cpu_time = round((time.time() - start_time)/n_replication,2)

print(f'AVG CPU time: {avg_cpu_time} seconds')
#print(f'Optimal solution answer: {case3_opt_sol}')
print(f'My optimal solution: {best_tnc}')
print(f'AVG optimal solution: {avg_tnc}')
#print(f'Gap (%): {gap}%')

Initial iteration is starting...
Global best value: 40736925359.2
Global best config: [9, 9, 33, 8, 33, 28, 33, 8, 9, 9, 33, 30, 28, 28, 8, 8, 8, 8, 8, 28, 30, 28, 8, 28, 9, 9, 9, 28, 30, 30, 28, 8, 33, 30, 30, 9, 9, 33, 30, 8, 30, 9, 30, 28, 8, 33, 8, 28, 30, 9, 33, 9, 9, 9, 28]
====================== REPLICATION 0 IS FINISHED ======================



Initial iteration is starting...
Global best value: 38285506511.6
Global best config: [4, 27, 31, 4, 27, 38, 27, 38, 4, 31, 27, 31, 38, 38, 27, 27, 36, 38, 27, 27, 27, 27, 31, 31, 31, 38, 27, 31, 4, 27, 31, 36, 36, 27, 4, 36, 31, 38, 4, 27, 36, 4, 4, 31, 36, 36, 27, 38, 36, 36, 36, 36, 27, 38, 31]
====================== REPLICATION 1 IS FINISHED ======================



Initial iteration is starting...
Global best value: 38957148371.2
Global best config: [39, 13, 13, 21, 21, 9, 34, 9, 9, 9, 13, 13, 13, 13, 21, 13, 39, 39, 21, 13, 21, 9, 39, 34, 9, 34, 21, 13, 9, 9, 34, 39, 39, 34, 39, 13, 21, 9, 39, 21, 21, 9, 34, 39, 34, 13, 9, 39, 21

# 5) TR n=55, p=5

In [27]:
%%time

start_time = time.time()

n_node=55
n_hub=7
swarm_size=100
epsilon=0.01
x_min=0
x_max = n_node*n_hub
flow_matrix = TR55_flow
cost_matrix = TR55_cost
theta = 0.1
inertia_factor = 1
C = 2

global_minimum_solution_list = []
replication = 0
n_replication = 10

while replication < n_replication:
    PSO_result = pso(n_node, n_hub, swarm_size, flow_matrix, cost_matrix, epsilon, theta, inertia_factor, C)
    global_minimum_solution_list.append(PSO_result)
    print(f'====================== REPLICATION {replication} IS FINISHED ======================\n')
    print('\n')
    replication += 1
    
case3_df = pd.DataFrame(global_minimum_solution_list, columns=['cost','node','unique_hub'])
avg_tnc = round(case3_df["cost"].mean(),2)
best_tnc = round(case3_df["cost"].min(),2)
#gap = round((best_tnc - case3_opt_sol)/best_tnc*100,2)
avg_cpu_time = round((time.time() - start_time)/n_replication,2)

print(f'AVG CPU time: {avg_cpu_time} seconds')
#print(f'Optimal solution answer: {case3_opt_sol}')
print(f'My optimal solution: {best_tnc}')
print(f'AVG optimal solution: {avg_tnc}')
#print(f'Gap (%): {gap}%')

Initial iteration is starting...
Global best value: 39211431403.8
Global best config: [14, 13, 14, 4, 34, 14, 32, 8, 13, 34, 51, 4, 13, 14, 8, 8, 13, 13, 51, 32, 14, 34, 4, 34, 32, 51, 32, 14, 13, 51, 14, 32, 13, 34, 14, 51, 34, 8, 8, 32, 4, 14, 4, 13, 32, 34, 32, 14, 51, 14, 51, 8, 4, 34, 4]
====================== REPLICATION 0 IS FINISHED ======================



Initial iteration is starting...
Global best value: 40969433038.2
Global best config: [31, 25, 28, 4, 7, 45, 7, 7, 7, 31, 7, 31, 52, 4, 31, 25, 25, 25, 25, 4, 45, 31, 45, 52, 25, 28, 31, 28, 45, 4, 31, 31, 7, 4, 52, 45, 25, 28, 4, 45, 45, 28, 31, 28, 45, 7, 25, 31, 31, 4, 31, 52, 4, 4, 28]
====================== REPLICATION 1 IS FINISHED ======================



Initial iteration is starting...
Global best value: 39231358857.6
Global best config: [1, 46, 7, 4, 4, 4, 7, 33, 13, 13, 24, 24, 13, 13, 7, 1, 33, 13, 7, 7, 46, 33, 46, 24, 33, 1, 46, 7, 1, 33, 33, 46, 33, 13, 1, 7, 13, 1, 46, 1, 33, 33, 46, 46, 46, 46, 1, 33, 46, 

# 6) TR81 n=81, p=5

In [31]:
%%time

start_time = time.time()

n_node=81
n_hub=5
swarm_size=100
epsilon=0.01
x_min=0
x_max = n_node*n_hub
flow_matrix = TR81_flow
cost_matrix = TR81_cost
theta = 0.1
inertia_factor = 1
C = 2

global_minimum_solution_list = []
replication = 0
n_replication = 10

while replication < n_replication:
    PSO_result = pso(n_node, n_hub, swarm_size, flow_matrix, cost_matrix, epsilon, theta, inertia_factor, C)
    global_minimum_solution_list.append(PSO_result)
    print(f'====================== REPLICATION {replication} IS FINISHED ======================\n')
    print('\n')
    replication += 1
    
case4_df = pd.DataFrame(global_minimum_solution_list, columns=['cost','node','unique_hub'])
avg_tnc = round(case4_df["cost"].mean(),2)
best_tnc = round(case4_df["cost"].min(),2)
#gap = round((best_tnc - case4_opt_sol)/best_tnc*100,2)
avg_cpu_time = round((time.time() - start_time)/n_replication,2)

print(f'AVG CPU time: {avg_cpu_time} seconds')
#print(f'Optimal solution answer: {case4_opt_sol}')
print(f'My optimal solution: {best_tnc}')
print(f'AVG optimal solution: {avg_tnc}')
print(f'Gap (%): {gap}%')

Initial iteration is starting...
Global best value: 76179184670.98
Global best config: [30, 30, 30, 30, 30, 57, 57, 30, 57, 57, 70, 63, 30, 30, 66, 57, 66, 57, 57, 70, 66, 70, 63, 63, 57, 66, 30, 66, 30, 30, 63, 57, 66, 63, 30, 30, 66, 57, 30, 57, 66, 66, 70, 70, 70, 57, 57, 70, 66, 63, 57, 30, 30, 30, 63, 30, 57, 63, 70, 57, 63, 70, 63, 57, 66, 66, 66, 70, 57, 70, 57, 57, 63, 57, 63, 30, 30, 63, 30, 57, 30]
====================== REPLICATION 0 IS FINISHED ======================



Initial iteration is starting...
Global best value: 71424328447.27
Global best config: [71, 18, 71, 56, 71, 6, 6, 6, 6, 18, 71, 56, 71, 6, 71, 56, 34, 18, 34, 34, 56, 34, 18, 34, 56, 6, 18, 34, 71, 71, 6, 18, 56, 34, 56, 18, 34, 34, 6, 6, 71, 6, 56, 71, 18, 18, 71, 34, 71, 71, 18, 6, 71, 71, 6, 56, 34, 34, 34, 6, 18, 56, 34, 6, 71, 34, 71, 56, 34, 56, 71, 34, 6, 34, 18, 6, 56, 34, 56, 56, 18]
====================== REPLICATION 1 IS FINISHED ======================



Initial iteration is starting...
Global be

# 7) TR81 n=81, p=7

In [32]:
%%time

start_time = time.time()

n_node=81
n_hub=7
swarm_size=100
epsilon=0.01
x_min=0
x_max = n_node*n_hub
flow_matrix = TR81_flow
cost_matrix = TR81_cost
theta = 0.1
inertia_factor = 1
C = 2

global_minimum_solution_list = []
replication = 0
n_replication = 10

while replication < n_replication:
    PSO_result = pso(n_node, n_hub, swarm_size, flow_matrix, cost_matrix, epsilon, theta, inertia_factor, C)
    global_minimum_solution_list.append(PSO_result)
    print(f'====================== REPLICATION {replication} IS FINISHED ======================\n')
    print('\n')
    replication += 1
    
case4_df = pd.DataFrame(global_minimum_solution_list, columns=['cost','node','unique_hub'])
avg_tnc = round(case4_df["cost"].mean(),2)
best_tnc = round(case4_df["cost"].min(),2)
#gap = round((best_tnc - case4_opt_sol)/best_tnc*100,2)
avg_cpu_time = round((time.time() - start_time)/n_replication,2)

print(f'AVG CPU time: {avg_cpu_time} seconds')
#print(f'Optimal solution answer: {case4_opt_sol}')
print(f'My optimal solution: {best_tnc}')
print(f'AVG optimal solution: {avg_tnc}')
print(f'Gap (%): {gap}%')

Initial iteration is starting...
Global best value: 74923185101.33
Global best config: [46, 46, 33, 6, 5, 6, 46, 46, 5, 5, 6, 5, 69, 16, 46, 16, 6, 52, 5, 52, 6, 6, 16, 16, 33, 33, 46, 6, 52, 5, 46, 52, 33, 33, 52, 46, 46, 16, 52, 46, 33, 69, 33, 46, 16, 46, 69, 52, 69, 52, 5, 52, 16, 52, 5, 69, 69, 33, 16, 33, 16, 33, 46, 52, 33, 33, 69, 5, 69, 69, 33, 69, 46, 69, 69, 33, 5, 46, 52, 16, 69]
====================== REPLICATION 0 IS FINISHED ======================



Initial iteration is starting...
Global best value: 73325522140.85
Global best config: [1, 1, 29, 59, 1, 21, 29, 40, 80, 66, 29, 1, 66, 21, 21, 59, 29, 66, 80, 59, 21, 1, 40, 66, 40, 40, 59, 66, 29, 66, 59, 29, 1, 21, 29, 21, 1, 21, 66, 40, 29, 40, 40, 59, 59, 21, 80, 80, 1, 40, 59, 29, 21, 59, 21, 1, 40, 80, 59, 66, 40, 29, 29, 40, 29, 66, 1, 21, 1, 1, 80, 80, 80, 59, 66, 29, 66, 29, 40, 80, 80]
====================== REPLICATION 1 IS FINISHED ======================



Initial iteration is starting...
Global best value: 729

# 8) RGP100 n=100, p=7

In [ ]:
%%time

start_time = time.time()

n_node=100
n_hub=7
swarm_size=100
#alpha = 0.2
epsilon=0.01
x_min=0
x_max = n_node*n_hub
flow_matrix = RGP100_flow
cost_matrix = RGP100_cost
theta = 0.1
inertia_factor = 1
C = 2

global_minimum_solution_list = []
replication = 0
n_replication = 10

while replication < n_replication:
    PSO_result = pso(n_node, n_hub, swarm_size, flow_matrix, cost_matrix, epsilon, theta, inertia_factor, C)
    global_minimum_solution_list.append(PSO_result)
    print(f'====================== REPLICATION {replication} IS FINISHED ======================\n')
    print('\n')
    replication += 1
    
case5_df = pd.DataFrame(global_minimum_solution_list, columns=['cost','node','unique_hub'])
avg_tnc = round(case5_df["cost"].mean(),2)
best_tnc = round(case5_df["cost"].min(),2)
#gap = round((best_tnc - case5_opt_sol)/best_tnc*100,2)
avg_cpu_time = round((time.time() - start_time)/n_replication,2)

print(f'AVG CPU time: {avg_cpu_time} seconds')
#print(f'Optimal solution answer: {case5_opt_sol}')
print(f'My optimal solution: {best_tnc}')
print(f'AVG optimal solution: {avg_tnc}')
#print(f'Gap (%): {gap}%')

Initial iteration is starting...
Global best value: 2197410945898.4
Global best config: [67, 39, 67, 20, 16, 67, 39, 64, 41, 59, 41, 64, 64, 16, 59, 16, 41, 16, 59, 20, 59, 41, 41, 16, 20, 20, 64, 39, 64, 16, 20, 67, 67, 20, 41, 20, 59, 41, 39, 64, 41, 16, 41, 41, 41, 59, 59, 20, 59, 67, 41, 67, 59, 67, 67, 16, 20, 59, 59, 20, 67, 16, 59, 64, 64, 64, 67, 59, 41, 67, 16, 20, 20, 41, 39, 20, 16, 67, 41, 41, 16, 59, 16, 59, 39, 64, 16, 41, 16, 67, 20, 59, 39, 41, 59, 39, 39, 41, 20, 20]
====================== REPLICATION 0 IS FINISHED ======================



Initial iteration is starting...
Global best value: 2192850075090.8
Global best config: [74, 36, 87, 53, 87, 87, 36, 36, 53, 74, 36, 71, 53, 36, 74, 74, 53, 74, 86, 86, 74, 87, 23, 86, 36, 86, 87, 36, 87, 71, 53, 74, 86, 87, 74, 36, 87, 87, 87, 87, 87, 87, 71, 71, 71, 74, 87, 36, 86, 71, 53, 23, 53, 74, 86, 36, 53, 87, 23, 71, 36, 23, 23, 36, 74, 87, 86, 53, 53, 71, 71, 86, 23, 74, 23, 74, 53, 71, 71, 74, 23, 87, 71, 36, 53, 86, 87,